### Import libraries

In [2]:
import arcpy
import pandas as pd 

### Make a feature class from the layer package

In [3]:
# Define file path to layer package and geodatabase; file suffix
FundingDist_lpkx = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Data_Restructure\FundingDistribution_CCIgdbAR24_prelim.lpkx"
project_gdb = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Data_Restructure\Data_Restructure.gdb"
suffix = "AR24"

# Define the target feature class path in the project geodatabase where you would like to export features to
Funding_Dist = f"{project_gdb}\\Funding_Dist_{suffix}"

# Make a feature layer from the layer package (.lpkx)
arcpy.management.MakeFeatureLayer(FundingDist_lpkx, "temp_layer")

# Clear any existing selection
arcpy.management.SelectLayerByAttribute("temp_layer", "CLEAR_SELECTION")

# Ensure all features will be included
count = int(arcpy.management.GetCount("temp_layer")[0])
print(f"Number of features in source: {count}")

# Export the feature layer to your project geodatabase
arcpy.management.CopyFeatures("temp_layer", Funding_Dist)

# Optional cleanup if needed (delete the temporary layer)
arcpy.management.Delete("temp_layer")

print(f"Feature class has been successfully exported to: {project_gdb}")

Number of features in source: 10278
Feature class has been successfully exported to: C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Data_Restructure\Data_Restructure.gdb


### Calculate fields for final layer


In [4]:
# PctBIPOC

BIPOC_fields = ["Hispanic", "AfricanAm", "NativeAm", "OtherMult", "AAPI"]
new_field = "PctBIPOC"

# Add the new field
if new_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, new_field, "DOUBLE")

# Use an UpdateCursor to calculate the sum of BIPOC percentages
with arcpy.da.UpdateCursor(Funding_Dist, BIPOC_fields + [new_field]) as cursor:
    for row in cursor:
        # Check for invalid values (None, -999, -1998) 
        if any(value in (None, -999, -1998) for value in row[:-1]):
            row[-1] = None  # Assign NULL if invalid values are present
        else:
            # Sum the values and cap at 100 if no invalid values
            total = sum(value for value in row[:-1])
            row[-1] = min(total, 100)
        cursor.updateRow(row)

print(f"Field '{new_field}' has been calculated with NULLs for invalid values.")


Field 'PctBIPOC' has been calculated with NULLs for invalid values.


In [5]:
# BIPOCMaj

classification_field = "BIPOCMaj"
new_field = "PctBIPOC"

# Add the new field
if classification_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, classification_field, "TEXT", field_length=10)

# Use an UpdateCursor to classify the rows
with arcpy.da.UpdateCursor(Funding_Dist, [new_field, classification_field]) as cursor:
    for row in cursor:
        if row[0] == 0 or row[0] is None:  # Set to NULL if PctBIPOC is 0 or null
            row[1] = None
        else:
            row[1] = "BIPOC" if row[0] > 0 else "White"
        cursor.updateRow(row)

print(f"Field '{classification_field}' has been populated with NULL for missing demographic information.")

Field 'BIPOCMaj' has been populated with NULL for missing demographic information.


In [6]:
# DemogMaj

fields_to_compare = ["Hispanic", "AfricanAm", "NativeAm", "OtherMult", "AAPI", "White"]
classification_field = "DemogMaj"

# Add the classification field
if classification_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, classification_field, "TEXT", field_length=50)

# Use an UpdateCursor to classify based on the greatest value
with arcpy.da.UpdateCursor(Funding_Dist, fields_to_compare + [classification_field]) as cursor:
    for row in cursor:
        # Create a dictionary of field names and their values
        field_values = {field: value for field, value in zip(fields_to_compare, row[:-1]) if value is not None}
        if field_values:  # Ensure there are non-null values
            # Find the field with the greatest value
            majority_field = max(field_values, key=field_values.get)
            row[-1] = majority_field  # Set the classification to the field name
        else:
            row[-1] = None  # Set to None if all values are null
        cursor.updateRow(row)

print(f"Field '{classification_field}' has been populated with NULL for missing demographic information.")

Field 'DemogMaj' has been populated with NULL for missing demographic information.


In [7]:
# NOPPGGRF
NOPPGGRF_fields = ["point_noppggrf_total", "Poly_NOPPGGRF_TOTAL", "CT_NOPPGGRF_TOTAL", "Line_NOPPGGRF_TOTAL"]
new_field = "NOPPGGRF"

# Add the new field 
if new_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, new_field, "DOUBLE")

# Use an UpdateCursor to calculate the sum of the fields
with arcpy.da.UpdateCursor(Funding_Dist, NOPPGGRF_fields + [new_field]) as cursor:
    for row in cursor:
        # Sum the values of the fields, treating None as 0
        total = sum(value if value is not None else 0 for value in row[:-1])  # Ignore the last value (new_field)
        row[-1] = total  # Set the sum in the new field
        cursor.updateRow(row)

print(f"Field '{new_field}' has been calculated.")

Field 'NOPPGGRF' has been calculated.


In [8]:
# PPGGRF
PPGGRF_fields = ["point_ppggrf_total", "Poly_PPGGRF_TOTAL", "CT_PPGGRF_TOTAL", "Line_PPGGRF_TOTAL"]
new_field = "PPGGRF"

# Add the new field 
if new_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, new_field, "DOUBLE")

# Use an UpdateCursor to calculate the sum of the fields
with arcpy.da.UpdateCursor(Funding_Dist, PPGGRF_fields + [new_field]) as cursor:
    for row in cursor:
        # Sum the values of the fields, treating None as 0
        total = sum(value if value is not None else 0 for value in row[:-1])  
        row[-1] = total  # Set the sum in the new field
        cursor.updateRow(row)

print(f"Field '{new_field}' has been calculated.")

Field 'PPGGRF' has been calculated.


In [9]:
# GGRF
GGRF_fields = ["point_ggrf_total", "Poly_GGRF_TOTAL", "CT_GGRF_TOTAL", "Line_GGRF_TOTAL"]
new_field = "GGRF"

# Add the new field 
if new_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, new_field, "DOUBLE")

# Use an UpdateCursor to calculate the sum of the fields
with arcpy.da.UpdateCursor(Funding_Dist, GGRF_fields + [new_field]) as cursor:
    for row in cursor:
        # Sum the values of the fields, treating None as 0
        total = sum(value if value is not None else 0 for value in row[:-1])  
        row[-1] = total  # Set the sum in the new field
        cursor.updateRow(row)

print(f"Field '{new_field}' has been calculated.")

Field 'GGRF' has been calculated.


In [10]:
# CCI_pct

input_field = "GGRF"
percentile_field = "CCI_pct"

# Add the new percentile field 
if percentile_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, percentile_field, "DOUBLE")

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Calculate percentiles as decimals
values = sorted(values)  # Sort values for percentile calculation
percentile_ranks = {value: i / (len(values) - 1) for i, value in enumerate(values)} 

# Step 3: Update the percentile field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, percentile_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values
            # Assign the rounded decimal percentile rank to the new field
            row[1] = round(percentile_ranks[row[0]], 2)
        else:
            row[1] = None  # Assign NULL for invalid values
        cursor.updateRow(row)

print(f"CCI Percentile values rounded to two decimal places have been calculated and stored in the field '{percentile_field}'.")


CCI Percentile values rounded to two decimal places have been calculated and stored in the field 'CCI_pct'.


In [11]:
# CCIdecile

input_field = "GGRF"
category_field = "CCIdecile" 

# Add the new category field 
if category_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, category_field, "TEXT", field_length=50) 

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Sort values and define categories
values = sorted(values)  # Sort values for decile calculation
categories = [
    "Lowest Percentile",
    ">10th-20th Percentile",
    ">20th-30th Percentile",
    ">30th-40th Percentile",
    ">40th-50th Percentile",
    ">50th-60th Percentile",
    ">60th-70th Percentile",
    ">70th-80th Percentile",
    ">80th-90th Percentile",
    "Highest Percentile"
]
category_mapping = {value: categories[int(i / (len(values) / 10))] for i, value in enumerate(values)}

# Step 3: Update the category field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, category_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values
            # Assign the category label to the new field
            row[1] = category_mapping[row[0]]
        else:
            row[1] = None  # Assign NULL for invalid values
        cursor.updateRow(row)

print(f"CCI deciles have been calculated and stored in the field '{category_field}'.")


CCI deciles have been calculated and stored in the field 'CCIdecile'.


In [12]:
# CCIquartile

input_field = "GGRF"
category_field = "CCIquartile" 

# Add the new category field 
if category_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, category_field, "TEXT", field_length=50) 

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Sort values and define categories
values = sorted(values)  # Sort values for quartile calculation
categories = [
    "Lowest Quartile",       # 0% to 25%
    ">25th-50th Quartile",   # 25% to 50%
    ">50th-75th Quartile",   # 50% to 75%
    "Highest Quartile"       # 75% to 100%
]
quartile_mapping = {value: categories[int(i / (len(values) / 4))] for i, value in enumerate(values)}

# Step 3: Update the category field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, category_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values
            # Assign the quartile label to the new field
            row[1] = quartile_mapping[row[0]]
        else:
            row[1] = None  # Assign NULL for invalid values
        cursor.updateRow(row)

print(f"CCI quartile categories have been calculated and stored in the field '{category_field}'.")


CCI quartile categories have been calculated and stored in the field 'CCIquartile'.


In [13]:
# CCIpc

input_field = "GGRF"
population_field = "Pop2020Rounded"
per_capita_field = "CCIpc"

# Add the new per capita field 
if per_capita_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, per_capita_field, "DOUBLE") 

# Calculate per capita funding
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, population_field, per_capita_field]) as cursor:
    for row in cursor:
        funding = row[0]
        population = row[1]

        # Check if population is valid and non-zero
        if population and population > 0:
            row[2] = funding / population  # Calculate per capita funding
        else:
            row[2] = None  # Assign NULL if population is 0 or None

        cursor.updateRow(row)

print(f"Per capita funding values have been calculated and stored in the field '{per_capita_field}'.")


Per capita funding values have been calculated and stored in the field 'CCIpc'.


In [14]:
# CCIpcPct

input_field = "CCIpc"
percentile_field = "CCIpcPct"

# Add the new percentile field
if percentile_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, percentile_field, "DOUBLE")

# Step 1: Extract values from the source field (ignoring None values)
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Ensure we have at least one valid value
if len(values) > 0:
    # Step 2: Calculate percentiles as decimals
    values = sorted(values)  # Sort values for percentile calculation
    percentile_ranks = {value: i / (len(values) - 1) for i, value in enumerate(values)}  # Decimal percentile rank

    # Step 3: Update the percentile field
    with arcpy.da.UpdateCursor(Funding_Dist, [input_field, percentile_field]) as cursor:
        for row in cursor:
            if row[0] is not None:  # Check for valid values in CCIpc
                # Assign the percentile rank as a decimal, rounded to two decimal places
                row[1] = round(percentile_ranks[row[0]], 2)
            else:
                row[1] = None  # Assign NULL for invalid values (None in CCIpc)
            cursor.updateRow(row)

    print(f"CCIpc Percentile values rounded to two decimal places have been calculated and stored in the field '{percentile_field}'.")
else:
    print("No valid values found in the input field.")


CCIpc Percentile values rounded to two decimal places have been calculated and stored in the field 'CCIpcPct'.


In [15]:
# CCIpcDecile

input_field = "CCIpc"
category_field = "CCIpcDecile" 

# Add the new category field
if category_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, category_field, "TEXT", field_length=50)

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Sort values and define categories
values = sorted(values)  # Sort values for decile calculation
categories = [
    "Lowest Percentile",
    ">10th-20th Percentile",
    ">20th-30th Percentile",
    ">30th-40th Percentile",
    ">40th-50th Percentile",
    ">50th-60th Percentile",
    ">60th-70th Percentile",
    ">70th-80th Percentile",
    ">80th-90th Percentile",
    "Highest Percentile"
]

# Calculate deciles and map to categories
category_mapping = {value: categories[int(i / (len(values) / 10))] for i, value in enumerate(values)}

# Step 3: Update the category field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, category_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values in CCIpc
            # Assign the appropriate category label
            row[1] = category_mapping[row[0]]
        else:
            row[1] = None  # Assign NULL if CCIpc is None
        cursor.updateRow(row)

print(f"CCI categories have been calculated and stored in the field '{category_field}'.")


CCI categories have been calculated and stored in the field 'CCIpcDecile'.


In [16]:
# CCIpcQuartile

input_field = "CCIpc"
category_field = "CCIpcQuartile"  

# Add the new category field if it doesn't exist
if category_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, category_field, "TEXT", field_length=50) 

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Sort values and define categories for quartiles
values = sorted(values)  # Sort values for quartile calculation
categories = [
    "Lowest Quartile",       # 0% to 25%
    ">25th-50th Quartile",   # 25% to 50%
    ">50th-75th Quartile",   # 50% to 75%
    "Highest Quartile"       # 75% to 100%
]

# Calculate quartiles and map to categories
quartile_mapping = {value: categories[int(i / (len(values) / 4))] for i, value in enumerate(values)}

# Step 3: Update the category field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, category_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values in CCIpc
            # Assign the appropriate quartile label
            row[1] = quartile_mapping[row[0]]
        else:
            row[1] = None  # Assign NULL if CCIpc is None
        cursor.updateRow(row)

print(f"CCI quartile categories have been calculated and stored in the field '{category_field}'.")


CCI quartile categories have been calculated and stored in the field 'CCIpcQuartile'.


In [17]:
# CES4range

input_field = "CIscoreP"
range_field = "CES4range" 

# Add the new range field
if range_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, range_field, "TEXT", field_length=50) 

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Sort values and define 5% range categories
values = sorted(values)  # Sort values
categories = [f"{i * 5}%-{(i + 1) * 5}%" for i in range(20)]  # 5% ranges from 0% to 100%

# Map values to 5% ranges
range_mapping = {}
bin_size = len(values) / 20  # Divide sorted values into 20 bins (5% each)
for i, value in enumerate(values):
    range_mapping[value] = categories[int(i // bin_size)]  # Assign range category

# Step 3: Update the range field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, range_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values in CIscoreP
            # Assign the appropriate range label
            row[1] = range_mapping[row[0]]
        else:
            row[1] = None  # Assign NULL if CIscoreP is None
        cursor.updateRow(row)

print(f"5% range categories have been calculated and stored in the field '{range_field}'.")


5% range categories have been calculated and stored in the field 'CES4range'.


In [18]:
# CES4decile

input_field = "CIscoreP"
decile_field = "CES4decile"  

# Add the new decile field 
if decile_field not in [f.name for f in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, decile_field, "TEXT", field_length=50)  

# Step 1: Extract values from the source field
values = []
with arcpy.da.SearchCursor(Funding_Dist, [input_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Exclude None values
            values.append(row[0])

# Step 2: Sort values and define 10% range categories
values = sorted(values)  # Sort values
categories = [f"{i * 10}%-{(i + 1) * 10}%" for i in range(10)]  # 10% ranges from 0% to 100%

# Map values to 10% ranges
range_mapping = {}
bin_size = len(values) / 10  # Divide sorted values into 10 bins (10% each)
for i, value in enumerate(values):
    range_mapping[value] = categories[int(i // bin_size)]  # Assign range category

# Step 3: Update the decile field
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, decile_field]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Check for valid values in CIscoreP
            # Assign the appropriate decile label
            row[1] = range_mapping[row[0]]
        else:
            row[1] = None  # Assign NULL if CIscoreP is None
        cursor.updateRow(row)

print(f"10% range categories have been calculated and stored in the field '{decile_field}'.")


10% range categories have been calculated and stored in the field 'CES4decile'.


In [19]:
# PP_simple
input_field = "Designatio"  
output_field = "PP_simple" 

# Add the new field 
if output_field not in [field.name for field in arcpy.ListFields(Funding_Dist)]:
    arcpy.AddField_management(Funding_Dist, output_field, "TEXT", field_length=100)

# Define the aggregation logic as a dictionary
category_mapping = {
    "DAC 1/2 Mile Neighbor: Low-Income Community": "DAC 1/2 Mile Neighbor: Low-Income Community",
    "Low-Income Community": "Low-Income Community",
    "Disadvantaged Community CES": "Disadvantaged Community",
    "Disadvantaged Community CES, Disadvantaged Community Tribal Land, Low-Income Community": "Disadvantaged Community",
    "Disadvantaged Community CES, Low-Income Community": "Disadvantaged Community",
    "Disadvantaged Community Tribal Land": "Disadvantaged Community",
    "Disadvantaged Community Tribal Land, Low-Income Community": "Disadvantaged Community",
    "DAC 1/2 Mile Neighbor: Low-Income Household Eligible": "Not a Priority Population Area: Low-Income Households are Eligible",
    "Not a Priority Population": "Not a Priority Population Area: Low-Income Households are Eligible"
}

# Update the new field based on the input field's values
with arcpy.da.UpdateCursor(Funding_Dist, [input_field, output_field]) as cursor:
    for row in cursor:
        input_value = row[0]  # Value from the input field
        # Determine the category based on the mapping
        category = category_mapping.get(input_value, "Unknown")  # Default to "Unknown" if no match
        row[1] = category  # Set the category in the output field
        cursor.updateRow(row)

print("Demographic categorization completed successfully.")


Demographic categorization completed successfully.


In [20]:
# List of fields to delete in preperation for program funding summing

fields_to_delete = [
    "Tract_1", "TotPop19", "Shape_Area_1"]

# Delete fields
try:
    arcpy.DeleteField_management(Funding_Dist, fields_to_delete)
    print("Fields deleted successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Fields deleted successfully.


In [21]:
# Create program specific funding fields 

Funding_Dist = f"{project_gdb}\\Funding_Dist_{suffix}"

# Define groups based on suffixes
field_groups = {
    "1": ["point_noppggrf_1", "point_ppggrf_1"],
    "2": ["point_noppggrf_2"],
    "3": ["point_noppggrf_3", "point_ppggrf_3"],
    "4": ["CT_NOPPGGRF_4", "CT_PPGGRF_4"],
    "5": ["point_noppggrf_5", "point_ppggrf_5"],
    "7": ["point_ppggrf_7", "Poly_PPGGRF_7"],
    "8": ["CT_NOPPGGRF_8", "CT_PPGGRF_8"],
    "9": ["CT_NOPPGGRF_9", "CT_PPGGRF_9"],
    "10": ["point_ppggrf_10"],
    "11": ["point_noppggrf_11", "point_ppggrf_11"],
    "14": ["point_noppggrf_14", "point_ppggrf_14"],
    "41": ["point_noppggrf_41", "point_ppggrf_41"],
    "45": ["point_noppggrf_45", "point_ppggrf_45"],
    "68": ["Line_NOPPGGRF_68", "Line_PPGGRF_68", "point_noppggrf_68", "point_ppggrf_68", "Poly_NOPPGGRF_68", "Poly_PPGGRF_68"],
    "72": ["point_noppggrf_72"],
    "76": ["point_noppggrf_76", "point_ppggrf_76"],
    "78": ["Line_NOPPGGRF_78", "Line_PPGGRF_78", "point_noppggrf_78", "point_ppggrf_78", "Poly_NOPPGGRF_78", "Poly_PPGGRF_78"],
    "80": ["CT_NOPPGGRF_80", "CT_PPGGRF_80"],
    "82": ["CT_NOPPGGRF_82", "CT_PPGGRF_82"],
    "86": ["point_noppggrf_86", "point_ppggrf_86", "Poly_NOPPGGRF_86"],
    "88": ["CT_NOPPGGRF_88", "CT_PPGGRF_88"],
    "90": ["CT_NOPPGGRF_90", "CT_PPGGRF_90"],
    "120": ["point_ppggrf_120"],
    "124": ["point_noppggrf_124", "point_ppggrf_124"],
    "128": ["CT_NOPPGGRF_128", "CT_PPGGRF_128"],
    "130": ["point_ppggrf_130"],
    "132": ["point_noppggrf_132", "point_ppggrf_132"],
    "134": ["point_noppggrf_134", "point_ppggrf_134"],
    "136": ["point_noppggrf_136", "point_ppggrf_136"],
    "138": ["point_noppggrf_138", "point_ppggrf_138"],
    "142": ["point_noppggrf_142", "point_ppggrf_142"],
    "148": ["point_noppggrf_148", "point_ppggrf_148"],
    "189": ["Poly_PPGGRF_189"],
    "191": ["point_noppggrf_191", "point_ppggrf_191"],
    "193": ["point_noppggrf_193", "point_ppggrf_193"],
    "235": ["point_noppggrf_235", "point_ppggrf_235"],
    "237": ["point_noppggrf_237", "point_ppggrf_237"],
    "239": ["point_noppggrf_239", "point_ppggrf_239"],
    "243": ["point_noppggrf_243"],
    "245": ["point_noppggrf_245", "point_ppggrf_245"],
    "247": ["point_noppggrf_247", "point_ppggrf_247"],
    "249": ["point_noppggrf_249"],
    "251": ["Poly_NOPPGGRF_251", "Poly_PPGGRF_251", "point_noppggrf_251", "point_ppggrf_251"],
    "253": ["point_noppggrf_253", "point_ppggrf_253"],
    "255": ["point_noppggrf_255"],
    "257": ["point_noppggrf_257", "point_ppggrf_257"],
    "259": ["point_noppggrf_259", "point_ppggrf_259"],
    "261": ["point_noppggrf_261", "point_ppggrf_261"],
    "263": ["point_noppggrf_263", "point_ppggrf_263"],
    "265": ["Poly_NOPPGGRF_265", "Poly_PPGGRF_265", "point_noppggrf_265", "point_ppggrf_265"],
    "267": ["point_ppggrf_267"],
    "269": ["point_ppggrf_269"],
    "271": ["point_ppggrf_271"],
    "329": ["CT_PPGGRF_329"],
    "331": ["point_noppggrf_331"],
    "333": ["point_noppggrf_333", "point_ppggrf_333"],
    "335": ["point_noppggrf_335", "point_ppggrf_335"],
    "337": ["CT_NOPPGGRF_337", "CT_PPGGRF_337"],
    "339": ["point_noppggrf_339", "point_ppggrf_339"],
    "341": ["point_noppggrf_341", "point_ppggrf_341"],
    "345": ["point_ppggrf_345"], 
    "471": ["point_noppggrf_471", "point_ppggrf_471"],
    "475": ["point_noppggrf_475", "point_ppggrf_475"],
    "477": ["point_noppggrf_477"],
    "479": ["point_noppggrf_479", "point_ppggrf_479"],
    "547": ["point_noppggrf_547", "point_ppggrf_547"],
    "549": ["point_noppggrf_549", "point_ppggrf_549"],
    "613": ["point_noppggrf_613", "point_ppggrf_613"],
    "615": ["point_noppggrf_615"],
    "617": ["point_noppggrf_617"],
    "619": ["point_noppggrf_619", "point_ppggrf_619"],
    "621": ["point_noppggrf_621", "point_ppggrf_621"],
    "690": ["point_noppggrf_690", "point_ppggrf_690"],
    "695": ["point_ppggrf_695", "Poly_PPGGRF_695"],
    "765": ["point_noppggrf_765", "point_ppggrf_765"],
    "837": ["CT_PPGGRF_837"],
    "839": ["point_noppggrf_839"],
    "981": ["point_noppggrf_981"]}

# Add a field for each group and calculate the sum of the grouped fields
for group, fields in field_groups.items():
    # Add a new field for the summed value of each group
    new_field_name = f"sum_group_{group}"
    arcpy.AddField_management(Funding_Dist, new_field_name, "DOUBLE")
    
    # Use an update cursor to calculate the sum for each record
    with arcpy.da.UpdateCursor(Funding_Dist, fields + [new_field_name]) as cursor:
        for row in cursor:
            # Sum the values of the fields in the group
            total_sum = sum([row[fields.index(field)] if row[fields.index(field)] is not None else 0 for field in fields])
            row[-1] = total_sum  # Assign the sum to the new field
            cursor.updateRow(row)

print("Fields have been grouped and summed successfully.")

Fields have been grouped and summed successfully.


### Check Program and GGRF Totals

In [22]:
# Check sum of all program funding from calculated fields

program_sum_fields = ["sum_group_1", "sum_group_2", "sum_group_3", "sum_group_4", "sum_group_5", "sum_group_7", "sum_group_8", "sum_group_9", 
          "sum_group_10", "sum_group_11", "sum_group_14", "sum_group_41", "sum_group_45", "sum_group_68", "sum_group_72", "sum_group_76", 
          "sum_group_78", "sum_group_80", "sum_group_82", "sum_group_86", "sum_group_88", "sum_group_90", "sum_group_120", "sum_group_124", 
          "sum_group_128", "sum_group_130", "sum_group_132", "sum_group_134", "sum_group_136", "sum_group_138", "sum_group_142", "sum_group_148",
          "sum_group_189", "sum_group_191", "sum_group_193", "sum_group_235", "sum_group_237", "sum_group_239", "sum_group_243", "sum_group_245", 
          "sum_group_247", "sum_group_249", "sum_group_251", "sum_group_253", "sum_group_255", "sum_group_257", "sum_group_259", 
          "sum_group_261", "sum_group_263", "sum_group_265", "sum_group_267", "sum_group_269", "sum_group_271", "sum_group_329", 
          "sum_group_331", "sum_group_333", "sum_group_335", "sum_group_337", "sum_group_339", "sum_group_341", "sum_group_345",
          "sum_group_471", "sum_group_475", "sum_group_477", "sum_group_479", "sum_group_547", "sum_group_549", "sum_group_613", 
          "sum_group_615", "sum_group_617", "sum_group_619", "sum_group_621", "sum_group_690", "sum_group_695", "sum_group_765",
          "sum_group_837", "sum_group_839", "sum_group_981"]


# Initialize a total variable for all fields
total_sum = 0

# Use a SearchCursor to iterate through the field values
with arcpy.da.SearchCursor(Funding_Dist, program_sum_fields) as cursor:
    for row in cursor:
        for value in row:
            if value is not None:  # Ensure the value is not NULL
                total_sum += value

# Round and print the result
total_sum = round(total_sum)
print(f"The total sum of all summed calculated program fields is: {total_sum:,}")

The total sum of all summed calculated program fields is: 11,011,906,653


In [23]:
# Check sum of all program funding from the input data fields (to check against the total from calculated fields above)

fields = ["CT_NOPPGGRF_128", "CT_NOPPGGRF_337", "CT_NOPPGGRF_4", "CT_NOPPGGRF_8", "CT_NOPPGGRF_80", "CT_NOPPGGRF_82", 
          "CT_NOPPGGRF_88", "CT_NOPPGGRF_9", "CT_NOPPGGRF_90", "CT_PPGGRF_128", "CT_PPGGRF_329", "CT_PPGGRF_337", "CT_PPGGRF_4", 
          "CT_PPGGRF_8", "CT_PPGGRF_80", "CT_PPGGRF_82", "CT_PPGGRF_837", "CT_PPGGRF_88", "CT_PPGGRF_9", "CT_PPGGRF_90", 
          "Line_NOPPGGRF_68", "Line_NOPPGGRF_78", "Line_PPGGRF_68", "Line_PPGGRF_78", "point_noppggrf_1", "point_noppggrf_11", 
          "point_noppggrf_124", "point_noppggrf_132", "point_noppggrf_134", "point_noppggrf_136", "point_noppggrf_138", 
          "point_noppggrf_14", "point_noppggrf_142", "point_noppggrf_148", "point_noppggrf_191", "point_noppggrf_193",
          "point_noppggrf_2", "point_noppggrf_235", "point_noppggrf_237", "point_noppggrf_239", "point_noppggrf_243", 
          "point_noppggrf_245", "point_noppggrf_247", "point_noppggrf_249", "point_noppggrf_251", "point_noppggrf_253", 
          "point_noppggrf_255", "point_noppggrf_257", "point_noppggrf_259", "point_noppggrf_261", "point_noppggrf_263", 
          "point_noppggrf_265", "point_noppggrf_3", "point_noppggrf_331", "point_noppggrf_333", "point_noppggrf_335", 
          "point_noppggrf_339", "point_noppggrf_341", "point_noppggrf_41", "point_noppggrf_45", "point_noppggrf_471", 
          "point_noppggrf_475", "point_noppggrf_477", "point_noppggrf_479", "point_noppggrf_5", "point_noppggrf_547", 
          "point_noppggrf_549", "point_noppggrf_613", "point_noppggrf_615", "point_noppggrf_617", "point_noppggrf_619", 
          "point_noppggrf_621", "point_noppggrf_68", "point_noppggrf_690", "point_noppggrf_72", "point_noppggrf_76", 
          "point_noppggrf_765", "point_noppggrf_78", "point_noppggrf_839", "point_noppggrf_86", "point_noppggrf_981", 
          "point_ppggrf_1", "point_ppggrf_10", "point_ppggrf_11", "point_ppggrf_120", "point_ppggrf_124", 
          "point_ppggrf_130", "point_ppggrf_132", "point_ppggrf_134", "point_ppggrf_136", "point_ppggrf_138", 
          "point_ppggrf_14", "point_ppggrf_142", "point_ppggrf_148", "point_ppggrf_191", "point_ppggrf_193", 
          "point_ppggrf_235", "point_ppggrf_237", "point_ppggrf_239", "point_ppggrf_245", "point_ppggrf_247",
          "point_ppggrf_251", "point_ppggrf_253", "point_ppggrf_257", "point_ppggrf_259", "point_ppggrf_261", 
          "point_ppggrf_263", "point_ppggrf_265", "point_ppggrf_267", "point_ppggrf_269", "point_ppggrf_271",
          "point_ppggrf_3", "point_ppggrf_333", "point_ppggrf_335", "point_ppggrf_339", "point_ppggrf_341", 
          "point_ppggrf_345", "point_ppggrf_41", "point_ppggrf_45", "point_ppggrf_471", "point_ppggrf_475",
          "point_ppggrf_479", "point_ppggrf_5", "point_ppggrf_547", "point_ppggrf_549", "point_ppggrf_613",
          "point_ppggrf_619", "point_ppggrf_621", "point_ppggrf_68", "point_ppggrf_690", "point_ppggrf_695",
          "point_ppggrf_7", "point_ppggrf_76", "point_ppggrf_765", "point_ppggrf_78", "point_ppggrf_86", 
          "Poly_NOPPGGRF_251", "Poly_NOPPGGRF_265", "Poly_NOPPGGRF_68", "Poly_NOPPGGRF_78", "Poly_NOPPGGRF_86", 
          "Poly_PPGGRF_189", "Poly_PPGGRF_251", "Poly_PPGGRF_265", "Poly_PPGGRF_68", "Poly_PPGGRF_695", "Poly_PPGGRF_7",
          "Poly_PPGGRF_78"]


# Initialize a total variable for all fields
total_sum = 0

# Use a SearchCursor to iterate through the field values
with arcpy.da.SearchCursor(Funding_Dist, fields) as cursor:
    for row in cursor:
        for value in row:
            if value is not None:  # Ensure the value is not NULL
                total_sum += value

# Round and print the result
total_sum = round(total_sum)
print(f"The total sum of all summed input program fields is: {total_sum:,}")

The total sum of all summed input program fields is: 11,011,906,653


In [24]:
# Calculate SUM of GGRF totals (to check against above program total)

# List of fields to sum
fields = ["GGRF", "PPGGRF", "NOPPGGRF"]

# Initialize a dictionary to store totals
totals = {field: 0 for field in fields}

# Use a SearchCursor to iterate through the field values
with arcpy.da.SearchCursor(Funding_Dist, fields) as cursor:
    for row in cursor:
        for field, value in zip(fields, row):
            if value is not None:  # Ensure the value is not NULL
                totals[field] += value

# Print results
for field, total in totals.items():
    total = round(total)
    print(f"The total summed calculated GGRF field '{field}' is: {total:,}")

The total summed calculated GGRF field 'GGRF' is: 11,011,786,903
The total summed calculated GGRF field 'PPGGRF' is: 7,605,701,813
The total summed calculated GGRF field 'NOPPGGRF' is: 3,406,085,090


In [25]:
# Calculate the SUM of GGRF input fields (to check agaisnt calculated GGRF totals above)

# Define field categories
NOPPGGRF_fields = ["point_noppggrf_total", "Poly_NOPPGGRF_TOTAL", "CT_NOPPGGRF_TOTAL", "Line_NOPPGGRF_TOTAL"]
PPGGRF_fields = ["point_ppggrf_total", "Poly_PPGGRF_TOTAL", "CT_PPGGRF_TOTAL", "Line_PPGGRF_TOTAL"]
GGRF_fields = ["point_ggrf_total", "Poly_GGRF_TOTAL", "CT_GGRF_TOTAL", "Line_GGRF_TOTAL"]

# Dictionary to store totals
totals = {"NOPPGGRF": 0, "PPGGRF": 0, "GGRF": 0}

# Sum values for each category
for category, fields in zip(totals.keys(), [NOPPGGRF_fields, PPGGRF_fields, GGRF_fields]):
    with arcpy.da.SearchCursor(Funding_Dist, fields) as cursor:
        for row in cursor:
            for value in row:
                if value is not None:  # Ensure value is not NULL
                    totals[category] += value

# Print results
for category, total in totals.items():
    total = round(total)
    print(f"The total summed calculated {category} field is: {total:,}")

The total summed calculated NOPPGGRF field is: 3,406,085,090
The total summed calculated PPGGRF field is: 7,605,701,813
The total summed calculated GGRF field is: 11,011,786,903


### Save 'wide' version of the feature layer

In [26]:
# Path to save the 'wide' version of the feature class
Funding_Dist_wide = f"{project_gdb}\\Funding_Dist_{suffix}_wide"

# Step 1: Create a copy of the feature class
try:
    arcpy.CopyFeatures_management(Funding_Dist, Funding_Dist_wide)
    print(f"Feature class copied successfully to {Funding_Dist_wide}")
except Exception as e:
    print(f"An error occurred while copying the feature class: {e}")

Feature class copied successfully to C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Data_Restructure\Data_Restructure.gdb\Funding_Dist_AR24_wide


In [27]:
# delete unneeded fields

# List of fields to delete for "wide"
fields_to_delete = [
    "ApproxLoc", "Shape_Leng", "ExclCTdata4GIS_ExclCTdata",
    "CT_NOPPGGRF_4", "CT_NOPPGGRF_8", "CT_NOPPGGRF_9", "CT_NOPPGGRF_80", "CT_NOPPGGRF_82",
    "CT_NOPPGGRF_88", "CT_NOPPGGRF_90", "CT_NOPPGGRF_128", "CT_NOPPGGRF_337", "CT_PPGGRF_4",
    "CT_PPGGRF_8", "CT_PPGGRF_9", "CT_PPGGRF_80", "CT_PPGGRF_82", "CT_PPGGRF_88", "CT_PPGGRF_90",
    "CT_PPGGRF_128", "CT_PPGGRF_329", "CT_PPGGRF_337", "CT_PPGGRF_837", "FID_CES4PP_ctAR24",
    "Poly_NOPPGGRF_68", "Poly_NOPPGGRF_78", "Poly_NOPPGGRF_86", "Poly_NOPPGGRF_251", "Poly_NOPPGGRF_265",
    "Poly_PPGGRF_7", "Poly_PPGGRF_68", "Poly_PPGGRF_78", "Poly_PPGGRF_189", "Poly_PPGGRF_251",
    "Poly_PPGGRF_265", "Poly_PPGGRF_695", "point_noppggrf_1", "point_noppggrf_2", "point_noppggrf_3",
    "point_noppggrf_5", "point_noppggrf_11", "point_noppggrf_14", "point_noppggrf_41", "point_noppggrf_45",
    "point_noppggrf_68", "point_noppggrf_72", "point_noppggrf_76", "point_noppggrf_78", "point_noppggrf_86",
    "point_noppggrf_124", "point_noppggrf_132", "point_noppggrf_134", "point_noppggrf_136", "point_noppggrf_138",
    "point_noppggrf_142", "point_noppggrf_148", "point_noppggrf_191", "point_noppggrf_193", "point_noppggrf_235",
    "point_noppggrf_237", "point_noppggrf_239", "point_noppggrf_243", "point_noppggrf_245", "point_noppggrf_247",
    "point_noppggrf_249", "point_noppggrf_251", "point_noppggrf_253", "point_noppggrf_255", "point_noppggrf_257",
    "point_noppggrf_259", "point_noppggrf_261", "point_noppggrf_263", "point_noppggrf_265", "point_noppggrf_331",
    "point_noppggrf_333", "point_noppggrf_335", "point_noppggrf_339", "point_noppggrf_341", "point_noppggrf_471",
    "point_noppggrf_475", "point_noppggrf_477", "point_noppggrf_479", "point_noppggrf_547", "point_noppggrf_549",
    "point_noppggrf_613", "point_noppggrf_615", "point_noppggrf_617", "point_noppggrf_619", "point_noppggrf_621",
    "point_noppggrf_690", "point_noppggrf_765", "point_noppggrf_839", "point_noppggrf_981", "point_ppggrf_1",
    "point_ppggrf_3", "point_ppggrf_5", "point_ppggrf_7", "point_ppggrf_10", "point_ppggrf_11", "point_ppggrf_14",
    "point_ppggrf_41", "point_ppggrf_45", "point_ppggrf_68", "point_ppggrf_76", "point_ppggrf_78", "point_ppggrf_86",
    "point_ppggrf_120", "point_ppggrf_124", "point_ppggrf_130", "point_ppggrf_132", "point_ppggrf_134",
    "point_ppggrf_136", "point_ppggrf_138", "point_ppggrf_142", "point_ppggrf_148", "point_ppggrf_191",
    "point_ppggrf_193", "point_ppggrf_235", "point_ppggrf_237", "point_ppggrf_239", "point_ppggrf_245",
    "point_ppggrf_247", "point_ppggrf_251", "point_ppggrf_253", "point_ppggrf_257", "point_ppggrf_259",
    "point_ppggrf_261", "point_ppggrf_263", "point_ppggrf_265", "point_ppggrf_267", "point_ppggrf_269",
    "point_ppggrf_271", "point_ppggrf_333", "point_ppggrf_335", "point_ppggrf_339", "point_ppggrf_341",
    "point_ppggrf_345", "point_ppggrf_471", "point_ppggrf_475", "point_ppggrf_479", "point_ppggrf_547",
    "point_ppggrf_549", "point_ppggrf_613", "point_ppggrf_619", "point_ppggrf_621", "point_ppggrf_690",
    "point_ppggrf_695", "point_ppggrf_765", "Point_Count", "Line_OBJECTID", "Line_NOPPGGRF_68", "Line_NOPPGGRF_78",
    "Line_PPGGRF_68", "Line_PPGGRF_78", "CT_NOPPGGRF_TOTAL", "CT_PPGGRF_TOTAL", "CT_GGRF_TOTAL", "Poly_NOPPGGRF_TOTAL",
    "Poly_PPGGRF_TOTAL", "Poly_GGRF_TOTAL", "Poly_GGRF_TOTAL", "point_noppggrf_total", "point_ppggrf_total", 
    "point_ggrf_total", "Line_NOPPGGRF_TOTAL", "Line_PPGGRF_TOTAL", "Line_GGRF_TOTAL", "SUM_GGRF_TOTAL", 
    "Area", "sum_POP20", "sum_Area_SQUAREKILOMETERS", "Polygon_Count", "ZIP", "PriorityPopulation", 
    "DACorLIC", "Census_Tract"
]

# Delete fields
try:
    arcpy.DeleteField_management(Funding_Dist_wide, fields_to_delete)
    print("Fields deleted successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Fields deleted successfully.


### Create 'Long' File Version

In [28]:
# add a wide record unique ID to the layer 

# Add a new field for the unique ID
unique_id_field = "UniqueID_wide"
arcpy.management.AddField(Funding_Dist, unique_id_field, "LONG")

# Calculate the unique ID based on OBJECTID
arcpy.management.CalculateField(Funding_Dist, unique_id_field, "!OBJECTID!", "PYTHON3")

print(f"Unique ID field '{unique_id_field}' added and populated successfully.")


Unique ID field 'UniqueID_wide' added and populated successfully.


In [29]:
#Create program specific sums for PP GGRF

# Define groups for pp total 
field_groups = {
    "1_pp": ["point_ppggrf_1"],
    "3_pp": ["point_ppggrf_3"],
    "4_pp": ["CT_PPGGRF_4"],
    "5_pp": ["point_ppggrf_5"],
    "7_pp": ["point_ppggrf_7", "Poly_PPGGRF_7"],
    "8_pp": ["CT_PPGGRF_8"],
    "9_pp": ["CT_PPGGRF_9"],
    "10_pp": ["point_ppggrf_10"],
    "11_pp": ["point_ppggrf_11"],
    "14_pp": ["point_ppggrf_14"],
    "41_pp": ["point_ppggrf_41"],
    "45_pp": ["point_ppggrf_45"],
    "68_pp": ["Line_PPGGRF_68", "point_ppggrf_68", "Poly_PPGGRF_68"],
    "76_pp": ["point_ppggrf_76"],
    "78_pp": ["Line_PPGGRF_78", "point_ppggrf_78", "Poly_PPGGRF_78"],
    "80_pp": ["CT_PPGGRF_80"],
    "82_pp": ["CT_PPGGRF_82"],
    "86_pp": ["point_ppggrf_86"],
    "88_pp": ["CT_PPGGRF_88"],
    "90_pp": ["CT_PPGGRF_90"],
    "120_pp": ["point_ppggrf_120"],
    "124_pp": ["point_ppggrf_124"],
    "128_pp": ["CT_PPGGRF_128"],
    "130_pp": ["point_ppggrf_130"],
    "132_pp": ["point_ppggrf_132"],
    "134_pp": ["point_ppggrf_134"],
    "136_pp": ["point_ppggrf_136"],
    "138_pp": ["point_ppggrf_138"],
    "142_pp": ["point_ppggrf_142"],
    "148_pp": ["point_ppggrf_148"],
    "189_pp": ["Poly_PPGGRF_189"],
    "191_pp": ["point_ppggrf_191"],
    "193_pp": ["point_ppggrf_193"],
    "235_pp": ["point_ppggrf_235"],
    "237_pp": ["point_ppggrf_237"],
    "239_pp": ["point_ppggrf_239"],
    "245_pp": ["point_ppggrf_245"],
    "247_pp": ["point_ppggrf_247"],
    "251_pp": ["Poly_PPGGRF_251", "point_ppggrf_251"],
    "253_pp": ["point_ppggrf_253"],
    "257_pp": ["point_ppggrf_257"],
    "259_pp": ["point_ppggrf_259"],
    "261_pp": ["point_ppggrf_261"],
    "263_pp": ["point_ppggrf_263"],
    "265_pp": ["Poly_PPGGRF_265", "point_ppggrf_265"],
    "267_pp": ["point_ppggrf_267"],
    "269_pp": ["point_ppggrf_269"],
    "271_pp": ["point_ppggrf_271"],
    "329_pp": ["CT_PPGGRF_329"],
    "333_pp": ["point_ppggrf_333"],
    "335_pp": ["point_ppggrf_335"],
    "337_pp": ["CT_PPGGRF_337"],
    "339_pp": ["point_ppggrf_339"],
    "341_pp": ["point_ppggrf_341"],
    "345_pp": ["point_ppggrf_345"], 
    "471_pp": ["point_ppggrf_471"],
    "475_pp": ["point_ppggrf_475"],
    "479_pp": ["point_ppggrf_479"],
    "547_pp": ["point_ppggrf_547"],
    "549_pp": ["point_ppggrf_549"],
    "613_pp": ["point_ppggrf_613"],
    "619_pp": ["point_ppggrf_619"],
    "621_pp": ["point_ppggrf_621"],
    "690_pp": ["point_ppggrf_690"],
    "695_pp": ["point_ppggrf_695", "Poly_PPGGRF_695"],
    "765_pp": ["point_ppggrf_765"],
    "837_pp": ["CT_PPGGRF_837"]}

# Add a field for each group and calculate the sum of the grouped fields
for group, fields in field_groups.items():
    # Add a new field for the summed value of each group
    new_field_name = f"sum_group_{group}"
    arcpy.AddField_management(Funding_Dist, new_field_name, "DOUBLE")
    
    # Use an update cursor to calculate the sum for each record
    with arcpy.da.UpdateCursor(Funding_Dist, fields + [new_field_name]) as cursor:
        for row in cursor:
            # Sum the values of the fields in the group
            total_sum = sum([row[fields.index(field)] if row[fields.index(field)] is not None else 0 for field in fields])
            row[-1] = total_sum  # Assign the sum to the new field
            cursor.updateRow(row)

print("Fields have been grouped and summed successfully.")

Fields have been grouped and summed successfully.


In [30]:
#Create program specific sums for NO PP GGRF

# Define groups for nopp total 
field_groups = {
    "1_nopp": ["point_noppggrf_1"],
    "2_nopp": ["point_noppggrf_2"],
    "3_nopp": ["point_noppggrf_3"],
    "4_nopp": ["CT_NOPPGGRF_4"],
    "5_nopp": ["point_noppggrf_5"],
    "8_nopp": ["CT_NOPPGGRF_8"],
    "9_nopp": ["CT_NOPPGGRF_9"],
    "11_nopp": ["point_noppggrf_11"],
    "14_nopp": ["point_noppggrf_14"],
    "41_nopp": ["point_noppggrf_41"],
    "45_nopp": ["point_noppggrf_45"],
    "68_nopp": ["Line_NOPPGGRF_68","point_noppggrf_68", "Poly_NOPPGGRF_68"],
    "72_nopp": ["point_noppggrf_72"],
    "76_nopp": ["point_noppggrf_76"],
    "78_nopp": ["Line_NOPPGGRF_78", "point_noppggrf_78", "Poly_NOPPGGRF_78"],
    "80_nopp": ["CT_NOPPGGRF_80"],
    "82_nopp": ["CT_NOPPGGRF_82"],
    "86_nopp": ["point_noppggrf_86", "Poly_NOPPGGRF_86"],
    "88_nopp": ["CT_NOPPGGRF_88"],
    "90_nopp": ["CT_NOPPGGRF_90"],
    "124_nopp": ["point_noppggrf_124"],
    "128_nopp": ["CT_NOPPGGRF_128"],
    "132_nopp": ["point_noppggrf_132"],
    "134_nopp": ["point_noppggrf_134"],
    "136_nopp": ["point_noppggrf_136"],
    "138_nopp": ["point_noppggrf_138"],
    "142_nopp": ["point_noppggrf_142"],
    "148_nopp": ["point_noppggrf_148"],
    "191_nopp": ["point_noppggrf_191"],
    "193_nopp": ["point_noppggrf_193"],
    "235_nopp": ["point_noppggrf_235"],
    "237_nopp": ["point_noppggrf_237"],
    "239_nopp": ["point_noppggrf_239"],
    "243_nopp": ["point_noppggrf_243"],
    "245_nopp": ["point_noppggrf_245"],
    "247_nopp": ["point_noppggrf_247"],
    "249_nopp": ["point_noppggrf_249"],
    "251_nopp": ["Poly_NOPPGGRF_251", "point_noppggrf_251"],
    "253_nopp": ["point_noppggrf_253"],
    "255_nopp": ["point_noppggrf_255"],
    "257_nopp": ["point_noppggrf_257"],
    "259_nopp": ["point_noppggrf_259"],
    "261_nopp": ["point_noppggrf_261"],
    "263_nopp": ["point_noppggrf_263"],
    "265_nopp": ["Poly_NOPPGGRF_265", "point_noppggrf_265"],
    "331_nopp": ["point_noppggrf_331"],
    "333_nopp": ["point_noppggrf_333"],
    "335_nopp": ["point_noppggrf_335"],
    "337_nopp": ["CT_NOPPGGRF_337"],
    "339_nopp": ["point_noppggrf_339"],
    "341_nopp": ["point_noppggrf_341"],
    "471_nopp": ["point_noppggrf_471"],
    "475_nopp": ["point_noppggrf_475"],
    "477_nopp": ["point_noppggrf_477"],
    "479_nopp": ["point_noppggrf_479"],
    "547_nopp": ["point_noppggrf_547"],
    "549_nopp": ["point_noppggrf_549"],
    "613_nopp": ["point_noppggrf_613"],
    "615_nopp": ["point_noppggrf_615"],
    "617_nopp": ["point_noppggrf_617"],
    "619_nopp": ["point_noppggrf_619"],
    "621_nopp": ["point_noppggrf_621"],
    "690_nopp": ["point_noppggrf_690"],
    "765_nopp": ["point_noppggrf_765"],
    "839_nopp": ["point_noppggrf_839"],
    "981_nopp": ["point_noppggrf_981"]}
               
# Add a field for each group and calculate the sum of the grouped fields
for group, fields in field_groups.items():
    # Add a new field for the summed value of each group
    new_field_name = f"sum_group_{group}"
    arcpy.AddField_management(Funding_Dist, new_field_name, "DOUBLE")
    
    # Use an update cursor to calculate the sum for each record
    with arcpy.da.UpdateCursor(Funding_Dist, fields + [new_field_name]) as cursor:
        for row in cursor:
            # Sum the values of the fields in the group
            total_sum = sum([row[fields.index(field)] if row[fields.index(field)] is not None else 0 for field in fields])
            row[-1] = total_sum  # Assign the sum to the new field
            cursor.updateRow(row)

print("Fields have been grouped and summed successfully.")

Fields have been grouped and summed successfully.


In [31]:
# restructure the data to the long format

# Path to save the 'long' version of the feature class
Funding_Dist_long = f"{project_gdb}\\Funding_Dist_{suffix}_long"

# List of funding columns
funding_columns = ["sum_group_1", "sum_group_2", "sum_group_3", "sum_group_4", "sum_group_5", "sum_group_7", "sum_group_8", "sum_group_9", 
          "sum_group_10", "sum_group_11", "sum_group_14", "sum_group_41", "sum_group_45", "sum_group_68", "sum_group_72", "sum_group_76", 
          "sum_group_78", "sum_group_80", "sum_group_82", "sum_group_86", "sum_group_88", "sum_group_90", "sum_group_120", "sum_group_124", 
          "sum_group_128", "sum_group_130", "sum_group_132", "sum_group_134", "sum_group_136", "sum_group_138", "sum_group_142", "sum_group_148",
          "sum_group_189", "sum_group_191", "sum_group_193", "sum_group_235", "sum_group_237", "sum_group_239", "sum_group_243", "sum_group_245", 
          "sum_group_247", "sum_group_249", "sum_group_251", "sum_group_253", "sum_group_255", "sum_group_257", "sum_group_259", 
          "sum_group_261", "sum_group_263", "sum_group_265", "sum_group_267", "sum_group_269", "sum_group_271", "sum_group_329", 
          "sum_group_331", "sum_group_333", "sum_group_335", "sum_group_337", "sum_group_339", "sum_group_341", "sum_group_345",
          "sum_group_471", "sum_group_475", "sum_group_477", "sum_group_479", "sum_group_547", "sum_group_549", "sum_group_613", 
          "sum_group_615", "sum_group_617", "sum_group_619", "sum_group_621", "sum_group_690", "sum_group_695", "sum_group_765",
          "sum_group_837", "sum_group_839", "sum_group_981"]

# Get all the fields in the input shapefile
all_fields = [f.name for f in arcpy.ListFields(Funding_Dist) if f.type not in ("Geometry", "OID")]

# Create a copy of the input feature class
arcpy.management.CopyFeatures(Funding_Dist, Funding_Dist_long)

# Add new fields to the output shapefile
arcpy.AddField_management(Funding_Dist_long, 'Program_Name', 'TEXT', field_length=100)
arcpy.AddField_management(Funding_Dist_long, 'Funding_Amount', 'DOUBLE')

# Open an InsertCursor for the output feature class
with arcpy.da.InsertCursor(Funding_Dist_long, all_fields + ['Program_Name', 'Funding_Amount', 'SHAPE@']) as insert_cursor:
    # Open a SearchCursor to iterate through the input shapefile
    with arcpy.da.SearchCursor(Funding_Dist, all_fields + funding_columns + ['SHAPE@']) as search_cursor:
        for row in search_cursor:
            # Extract values for fields that should remain unchanged
            original_values = list(row[:len(all_fields)])
            geometry = row[-1]  # Extract the geometry
            
            # Iterate through each funding column
            for idx, funding_column in enumerate(funding_columns, start=len(all_fields)):
                funding_amount = row[idx]  # Access the funding amount from the corresponding column
                
                # Insert a new row if the funding amount is greater than 0
                if funding_amount > 0:
                    insert_cursor.insertRow(original_values + [funding_column, funding_amount, geometry])

print("Restructuring completed.")

Restructuring completed.


In [32]:
# check program total sum of individual summed fields for long version 

# Define the columns to sum
funding_columns = [
    "sum_group_1", "sum_group_2", "sum_group_3", "sum_group_4", "sum_group_5", "sum_group_7", "sum_group_8", "sum_group_9", 
    "sum_group_10", "sum_group_11", "sum_group_14", "sum_group_41", "sum_group_45", "sum_group_68", "sum_group_72", "sum_group_76", 
    "sum_group_78", "sum_group_80", "sum_group_82", "sum_group_86", "sum_group_88", "sum_group_90", "sum_group_120", "sum_group_124", 
    "sum_group_128", "sum_group_130", "sum_group_132", "sum_group_134", "sum_group_136", "sum_group_138", "sum_group_142", "sum_group_148",
    "sum_group_189", "sum_group_191", "sum_group_193", "sum_group_235", "sum_group_237", "sum_group_239", "sum_group_243", "sum_group_245", 
    "sum_group_247", "sum_group_249", "sum_group_251", "sum_group_253", "sum_group_255", "sum_group_257", "sum_group_259", 
    "sum_group_261", "sum_group_263", "sum_group_265", "sum_group_267", "sum_group_269", "sum_group_271", "sum_group_329", 
    "sum_group_331", "sum_group_333", "sum_group_335", "sum_group_337", "sum_group_339", "sum_group_341", "sum_group_345",
    "sum_group_471", "sum_group_475", "sum_group_477", "sum_group_479", "sum_group_547", "sum_group_549", "sum_group_613", 
    "sum_group_615", "sum_group_617", "sum_group_619", "sum_group_621", "sum_group_690", "sum_group_695", "sum_group_765",
    "sum_group_837", "sum_group_839", "sum_group_981"
]

# Initialize total sum
total_sum = 0

# Use a search cursor to iterate over the records and sum the values
with arcpy.da.SearchCursor(Funding_Dist, funding_columns) as cursor:
    for row in cursor:
        for value in row:
            if value is not None:  # Ensure value is not NULL
                total_sum += value

# Print the final summed total
print(f"The total summed calculated funding field is: {total_sum:,}")

The total summed calculated funding field is: 11,011,906,652.860317


In [33]:
# check sum of final program total column

# Define the column to sum
funding_column = "Funding_Amount"

# Initialize total sum
total_sum = 0

# Use a search cursor to iterate over the records and sum the values
with arcpy.da.SearchCursor(Funding_Dist_long, [funding_column]) as cursor:
    for row in cursor:
        if row[0] is not None:  # Ensure value is not NULL
            total_sum += row[0]

# Print the final summed total
print(f"The total summed calculated funding amount is: {total_sum:,}")

The total summed calculated funding amount is: 11,011,906,652.860317


In [34]:
# Delete initial rows from "wide" format 

# Open an UpdateCursor to delete rows that have multiple inputs in program funding columns
with arcpy.da.UpdateCursor(Funding_Dist_long, ['Program_Name']) as cursor:
    for row in cursor:
        # Check if Program_Name is null or empty
        if row[0] is None or row[0] == "":
            cursor.deleteRow()

print("Rows with null values in Program_Name have been deleted.")

Rows with null values in Program_Name have been deleted.


In [35]:
# Delete the individual funding columns
for column in funding_columns:
    if arcpy.ListFields(Funding_Dist_long, column): 
        arcpy.DeleteField_management(Funding_Dist_long, column)

# Print completion message
print("All specified funding columns have been processed and deleted.")

All specified funding columns have been processed and deleted.


In [36]:
# delete additional unneeded fields

# List of fields to delete for "long" version
fields_to_delete = [
    "ApproxLoc", "Shape_Leng", "ExclCTdata4GIS_ExclCTdata",
    "CT_NOPPGGRF_4", "CT_NOPPGGRF_8", "CT_NOPPGGRF_9", "CT_NOPPGGRF_80", "CT_NOPPGGRF_82",
    "CT_NOPPGGRF_88", "CT_NOPPGGRF_90", "CT_NOPPGGRF_128", "CT_NOPPGGRF_337", "CT_PPGGRF_4",
    "CT_PPGGRF_8", "CT_PPGGRF_9", "CT_PPGGRF_80", "CT_PPGGRF_82", "CT_PPGGRF_88", "CT_PPGGRF_90",
    "CT_PPGGRF_128", "CT_PPGGRF_329", "CT_PPGGRF_337", "CT_PPGGRF_837", "FID_CES4PP_ctAR24",
    "Poly_NOPPGGRF_68", "Poly_NOPPGGRF_78", "Poly_NOPPGGRF_86", "Poly_NOPPGGRF_251", "Poly_NOPPGGRF_265",
    "Poly_PPGGRF_7", "Poly_PPGGRF_68", "Poly_PPGGRF_78", "Poly_PPGGRF_189", "Poly_PPGGRF_251",
    "Poly_PPGGRF_265", "Poly_PPGGRF_695", "point_noppggrf_1", "point_noppggrf_2", "point_noppggrf_3",
    "point_noppggrf_5", "point_noppggrf_11", "point_noppggrf_14", "point_noppggrf_41", "point_noppggrf_45",
    "point_noppggrf_68", "point_noppggrf_72", "point_noppggrf_76", "point_noppggrf_78", "point_noppggrf_86",
    "point_noppggrf_124", "point_noppggrf_132", "point_noppggrf_134", "point_noppggrf_136", "point_noppggrf_138",
    "point_noppggrf_142", "point_noppggrf_148", "point_noppggrf_191", "point_noppggrf_193", "point_noppggrf_235",
    "point_noppggrf_237", "point_noppggrf_239", "point_noppggrf_243", "point_noppggrf_245", "point_noppggrf_247",
    "point_noppggrf_249", "point_noppggrf_251", "point_noppggrf_253", "point_noppggrf_255", "point_noppggrf_257",
    "point_noppggrf_259", "point_noppggrf_261", "point_noppggrf_263", "point_noppggrf_265", "point_noppggrf_331",
    "point_noppggrf_333", "point_noppggrf_335", "point_noppggrf_339", "point_noppggrf_341", "point_noppggrf_471",
    "point_noppggrf_475", "point_noppggrf_477", "point_noppggrf_479", "point_noppggrf_547", "point_noppggrf_549",
    "point_noppggrf_613", "point_noppggrf_615", "point_noppggrf_617", "point_noppggrf_619", "point_noppggrf_621",
    "point_noppggrf_690", "point_noppggrf_765", "point_noppggrf_839", "point_noppggrf_981", "point_ppggrf_1",
    "point_ppggrf_3", "point_ppggrf_5", "point_ppggrf_7", "point_ppggrf_10", "point_ppggrf_11", "point_ppggrf_14",
    "point_ppggrf_41", "point_ppggrf_45", "point_ppggrf_68", "point_ppggrf_76", "point_ppggrf_78", "point_ppggrf_86",
    "point_ppggrf_120", "point_ppggrf_124", "point_ppggrf_130", "point_ppggrf_132", "point_ppggrf_134",
    "point_ppggrf_136", "point_ppggrf_138", "point_ppggrf_142", "point_ppggrf_148", "point_ppggrf_191",
    "point_ppggrf_193", "point_ppggrf_235", "point_ppggrf_237", "point_ppggrf_239", "point_ppggrf_245",
    "point_ppggrf_247", "point_ppggrf_251", "point_ppggrf_253", "point_ppggrf_257", "point_ppggrf_259",
    "point_ppggrf_261", "point_ppggrf_263", "point_ppggrf_265", "point_ppggrf_267", "point_ppggrf_269",
    "point_ppggrf_271", "point_ppggrf_333", "point_ppggrf_335", "point_ppggrf_339", "point_ppggrf_341",
    "point_ppggrf_345", "point_ppggrf_471", "point_ppggrf_475", "point_ppggrf_479", "point_ppggrf_547",
    "point_ppggrf_549", "point_ppggrf_613", "point_ppggrf_619", "point_ppggrf_621", "point_ppggrf_690",
    "point_ppggrf_695", "point_ppggrf_765", "Point_Count", "Line_OBJECTID", "Line_NOPPGGRF_68", "Line_NOPPGGRF_78",
    "Line_PPGGRF_68", "Line_PPGGRF_78", "NOPPGGRF", "PPGGRF", "GGRF", "CT_NOPPGGRF_TOTAL", "CT_PPGGRF_TOTAL", 
    "CT_GGRF_TOTAL", "Poly_NOPPGGRF_TOTAL", "Poly_PPGGRF_TOTAL", "Poly_GGRF_TOTAL", "point_noppggrf_total",
    "point_ppggrf_total", "point_ggrf_total", "Line_NOPPGGRF_TOTAL", "Line_PPGGRF_TOTAL", "Line_GGRF_TOTAL",
    "SUM_GGRF_TOTAL", "Area", "sum_POP20", "sum_Area_SQUAREKILOMETERS", "Polygon_Count", "ZIP", "PriorityPopulation",
    "DACorLIC", "Census_Tract", "CCI_pct", "CCIdecile", "CCIquartile", "CCIpcPct", "CCIpcDecile", "CCIpcQuartile", 
    "CCIpc"]

# Delete fields
try:
    arcpy.DeleteField_management(Funding_Dist_long, fields_to_delete)
    print("Fields deleted successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Fields deleted successfully.


In [37]:
# generate one PP program specific total field  

# Define the table or feature class
table = "Funding_Dist_AR24_long"

# Identify field names
fields = [f.name for f in arcpy.ListFields(table)]

# The column that contains the reference values (text like "sum_group_239")
reference_field = "Program_Name"

# Define the output column for storing matched values
output_field = "pp_funding"

# Ensure the output field exists
if output_field not in fields:
    arcpy.AddField_management(table, output_field, "DOUBLE")  # Adjust field type if needed

# Identify possible matching columns (those that end with "_pp")
matching_columns = {f: i for i, f in enumerate(fields) if f.endswith("_pp")}

# Ensure we're using correct field names in the cursor
cursor_fields = [reference_field, output_field] + list(matching_columns.keys())

# Update the table
with arcpy.da.UpdateCursor(table, cursor_fields) as cursor:
    for row in cursor:
        reference_value = row[0]  # Value from Program_Name (e.g., "sum_group_239")

        if reference_value:
            reference_str = str(reference_value).strip()  # Convert to string & remove spaces
            target_column = reference_str + "_pp"  # Construct expected column name

            if target_column in matching_columns:
                col_index = cursor_fields.index(target_column)  # Get correct index in cursor
                row[1] = row[col_index]  # Assign the value to pp_funding

        cursor.updateRow(row)

print("✅ Processing complete!")

✅ Processing complete!


In [38]:
# generate one noPP program specific total field 

# Define the table or feature class
table = "Funding_Dist_AR24_long"

# Identify field names
fields = [f.name for f in arcpy.ListFields(table)]

# The column that contains the reference values (text like "sum_group_239")
reference_field = "Program_Name"

# Define the output column for storing matched values
output_field = "nopp_funding"

# Ensure the output field exists
if output_field not in fields:
    arcpy.AddField_management(table, output_field, "DOUBLE")  # Adjust field type if needed

# Identify possible matching columns (those that end with "_pp")
matching_columns = {f: i for i, f in enumerate(fields) if f.endswith("_nopp")}

# Ensure we're using correct field names in the cursor
cursor_fields = [reference_field, output_field] + list(matching_columns.keys())

# Update the table
with arcpy.da.UpdateCursor(table, cursor_fields) as cursor:
    for row in cursor:
        reference_value = row[0]  # Value from Program_Name (e.g., "sum_group_239")

        if reference_value:
            reference_str = str(reference_value).strip()  # Convert to string & remove spaces
            target_column = reference_str + "_nopp"  # Construct expected column name

            if target_column in matching_columns:
                col_index = cursor_fields.index(target_column)  # Get correct index in cursor
                row[1] = row[col_index]  # Assign the value to nopp_funding

        cursor.updateRow(row)

print("✅ Processing complete!")

✅ Processing complete!


In [39]:
#Check the PP and no PP column totals

# Define the table or feature class
table = "Funding_Dist_AR24_long"

# Initialize sum variables
pp_funding_sum = sum(row[0] if row[0] is not None else 0 for row in arcpy.da.SearchCursor(table, ["pp_funding"]))
nopp_funding_sum = sum(row[0] if row[0] is not None else 0 for row in arcpy.da.SearchCursor(table, ["nopp_funding"]))

# Round the sums to 2 decimal places and format with commas
pp_funding_sum_formatted = "{:,.2f}".format(pp_funding_sum)
nopp_funding_sum_formatted = "{:,.2f}".format(nopp_funding_sum)

# Print the sums with commas and rounding
print(f"✅ Total pp_funding: {pp_funding_sum_formatted}")
print(f"✅ Total nopp_funding: {nopp_funding_sum_formatted}")


✅ Total pp_funding: 7,605,727,062.94
✅ Total nopp_funding: 3,406,179,589.92


In [40]:
# delete PP and no PP individual program columns

fields_to_delete = [
    "sum_group_1_pp", "sum_group_3_pp", "sum_group_4_pp", "sum_group_5_pp", "sum_group_7_pp",
    "sum_group_8_pp", "sum_group_9_pp", "sum_group_10_pp", "sum_group_11_pp", "sum_group_14_pp",
    "sum_group_41_pp", "sum_group_45_pp", "sum_group_68_pp", "sum_group_76_pp", "sum_group_78_pp",
    "sum_group_80_pp", "sum_group_82_pp", "sum_group_86_pp", "sum_group_88_pp", "sum_group_90_pp", "sum_group_120_pp",
    "sum_group_124_pp", "sum_group_128_pp", "sum_group_130_pp", "sum_group_132_pp", "sum_group_134_pp",
    "sum_group_136_pp", "sum_group_138_pp", "sum_group_142_pp", "sum_group_148_pp", "sum_group_189_pp",
    "sum_group_191_pp", "sum_group_193_pp", "sum_group_235_pp", "sum_group_237_pp", "sum_group_239_pp",
    "sum_group_245_pp", "sum_group_247_pp", "sum_group_251_pp", "sum_group_253_pp", "sum_group_257_pp",
    "sum_group_259_pp", "sum_group_261_pp", "sum_group_263_pp", "sum_group_265_pp", "sum_group_267_pp",
    "sum_group_269_pp", "sum_group_271_pp", "sum_group_329_pp", "sum_group_333_pp", "sum_group_335_pp",
    "sum_group_337_pp", "sum_group_339_pp", "sum_group_341_pp", "sum_group_345_pp", "sum_group_471_pp",
    "sum_group_475_pp", "sum_group_479_pp", "sum_group_547_pp", "sum_group_549_pp", "sum_group_613_pp",
    "sum_group_619_pp", "sum_group_621_pp", "sum_group_690_pp", "sum_group_695_pp", "sum_group_765_pp",
    "sum_group_837_pp", "sum_group_1_nopp", "sum_group_2_nopp", "sum_group_3_nopp", "sum_group_4_nopp",
    "sum_group_5_nopp", "sum_group_8_nopp", "sum_group_9_nopp", "sum_group_11_nopp", "sum_group_14_nopp",
    "sum_group_41_nopp", "sum_group_45_nopp", "sum_group_68_nopp", "sum_group_72_nopp", "sum_group_76_nopp",
    "sum_group_78_nopp", "sum_group_80_nopp", "sum_group_82_nopp", "sum_group_86_nopp", "sum_group_88_nopp",
    "sum_group_90_nopp", "sum_group_124_nopp", "sum_group_128_nopp", "sum_group_132_nopp", "sum_group_134_nopp",
    "sum_group_136_nopp", "sum_group_138_nopp", "sum_group_142_nopp", "sum_group_148_nopp", "sum_group_191_nopp",
    "sum_group_193_nopp", "sum_group_235_nopp", "sum_group_237_nopp", "sum_group_239_nopp", "sum_group_243_nopp",
    "sum_group_245_nopp", "sum_group_247_nopp", "sum_group_249_nopp", "sum_group_251_nopp", "sum_group_253_nopp",
    "sum_group_255_nopp", "sum_group_257_nopp", "sum_group_259_nopp", "sum_group_261_nopp", "sum_group_263_nopp",
    "sum_group_265_nopp", "sum_group_331_nopp", "sum_group_333_nopp", "sum_group_335_nopp", "sum_group_337_nopp",
    "sum_group_339_nopp", "sum_group_341_nopp", "sum_group_471_nopp", "sum_group_475_nopp", "sum_group_477_nopp",
    "sum_group_479_nopp", "sum_group_547_nopp", "sum_group_549_nopp", "sum_group_613_nopp", "sum_group_615_nopp",
    "sum_group_617_nopp", "sum_group_619_nopp", "sum_group_621_nopp", "sum_group_690_nopp", "sum_group_695_nopp",
    "sum_group_765_nopp", "sum_group_839_nopp", "sum_group_981_nopp"
]

# Delete fields
try:
    arcpy.DeleteField_management(Funding_Dist_long, fields_to_delete)
    print("Fields deleted successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Fields deleted successfully.


In [41]:
# Add demographic percentage columns
demographic_columns = ["Hispanic", "White", "AfricanAm", "NativeAm", "OtherMult", "PctBIPOC", "AAPI"]

# Add new fields to hold the range label for each demographic column
for column in demographic_columns:
    range_field_name = f"{column}_Range"
    arcpy.AddField_management(Funding_Dist_long, range_field_name, "TEXT", field_length=20)

# Populate the new fields with the appropriate range label
with arcpy.da.UpdateCursor(Funding_Dist_long, demographic_columns + [f"{col}_Range" for col in demographic_columns]) as cursor:
    for row in cursor:
        for col_index, column in enumerate(demographic_columns):
            percent_value = row[col_index]
            if percent_value is not None:
                # Determine the range the value falls into
                for i in range(1, 11):
                    range_start = (i - 1) * 10
                    range_end = i * 10
                    if range_start <= percent_value < range_end or (i == 10 and percent_value == 100): 
                        range_label = f"{range_start}-{range_end}%"
                        range_field_index = len(demographic_columns) + col_index
                        row[range_field_index] = range_label
                        break
            else:
                # If the value is None, leave the range field blank
                range_field_index = len(demographic_columns) + col_index
                row[range_field_index] = None
        cursor.updateRow(row)

print("Demographic range fields updated successfully.")

Demographic range fields updated successfully.


In [46]:
# Update the per capita column for "long" version 

input_field = "Funding_Amount"
population_field = "Pop2020Rounded"
per_capita_field = "CCIpc_prgm"

# Add the new per capita field 
if per_capita_field not in [f.name for f in arcpy.ListFields(Funding_Dist_long)]:
    arcpy.AddField_management(Funding_Dist_long, per_capita_field, "DOUBLE") 

# Calculate per capita funding
with arcpy.da.UpdateCursor(Funding_Dist_long, [input_field, population_field, per_capita_field]) as cursor:
    for row in cursor:
        funding = row[0]
        population = row[1]

        # Check if population is valid and non-zero
        if population and population > 0:
            row[2] = funding / population  # Calculate per capita funding
        else:
            row[2] = None  # Assign NULL if population is 0 or None

        cursor.updateRow(row)

print(f"Per capita funding values have been calculated and stored in the field '{per_capita_field}'.")

Per capita funding values have been calculated and stored in the field 'CCIpc_prgm'.


In [47]:
# Map old names in Program_Name to new descriptive names
column_mapping = {
    "sum_group_1": "Dairy Digester Research and Development Program",
    "sum_group_2": "Renewable and Alternative Fuels",
    "sum_group_3": "State Water Efficiency and Enhancement Program",
    "sum_group_4": "Clean Vehicle Rebate Project",
    "sum_group_5": "Clean Truck and Bus Vouchers (HVIP)",
    "sum_group_7": "Clean Mobility Options",
    "sum_group_8": "Clean Cars 4 All",
    "sum_group_9": "Financing Assistance for Lower-Income Consumers",
    "sum_group_10": "Advanced Technology Demonstration and Pilot Projects",
    "sum_group_11": "Zero-Emission Truck and Bus Pilot",
    "sum_group_14": "Sustainable Agricultural Lands Conservation Program",
    "sum_group_41": "Affordable Housing and Sustainable Communities Program",
    "sum_group_45": "Urban and Community Forestry",
    "sum_group_68": "Low Carbon Transit Operations Program",
    "sum_group_72": "State Water Project Turbines",
    "sum_group_76": "Wetlands and Watershed Restoration",
    "sum_group_78": "Transit and Intercity Rail Capital Program",
    "sum_group_80": "Single-Family Solar Photovoltaics (PV)",
    "sum_group_82": "Water-Energy Grant Program",
    "sum_group_86": "Forest Health Program",
    "sum_group_88": "Single-Family Energy Efficiency and Solar PV",
    "sum_group_90": "Multi-Family Energy Efficiency and Renewables",
    "sum_group_120": "Agricultural Worker Vanpools",
    "sum_group_124": "Rural School Bus Pilot Projects",
    "sum_group_128": "Woodsmoke Reduction Program",
    "sum_group_130": "Active Transportation Program",
    "sum_group_132": "Organics and Recycling Loans",
    "sum_group_134": "Recycled Fiber, Plastic, and Glass Grant Program",
    "sum_group_136": "Food Waste Prevention and Rescue Grants",
    "sum_group_138": "Urban Greening Program",
    "sum_group_142": "Transformative Climate Communities (Community)",
    "sum_group_148": "Organics Grants",
    "sum_group_189": "Community Solar Pilot",
    "sum_group_191": "Alternative Manure Management Program",
    "sum_group_193": "Healthy Soils Program",
    "sum_group_235": "Community Air Protection Incentives",
    "sum_group_237": "Funding Agricultural Replacement Measures for Emission Reductions Program",
    "sum_group_239": "Coastal Resilience Planning",
    "sum_group_243": "Climate Change Adaptation and Coastal Resilience Planning",
    "sum_group_245": "Fire Prevention Program",
    "sum_group_247": "Wildfire Prevention Grants Program",
    "sum_group_249": "Wildfire Response and Readiness",
    "sum_group_251": "Climate Adaptation and Resiliency Program",
    "sum_group_253": "Climate Ready Program",
    "sum_group_255": "Climate Change Research Program",
    "sum_group_257": "Food Production Investment Program",
    "sum_group_259": "Renewable Energy for Agriculture Program",
    "sum_group_261": "Training and Workforce Development Program",
    "sum_group_263": "Clean Off Road Equipment Voucher Incentive Project",
    "sum_group_265": "Community Air Grants",
    "sum_group_267": "Zero-and Near Zero-Emission Freight Facilities Project",
    "sum_group_269": "Advanced Technology Demonstration and Pilot Projects",
    "sum_group_271": "Advanced Technology Demonstration and Pilot Projects",
    "sum_group_329": "Transformative Climate Communities (Household)",
    "sum_group_331": "Prescribed Fire and Smoke Monitoring Program",
    "sum_group_333": "Forest Carbon Plan Implementation",
    "sum_group_335": "Low-Carbon Fuel Production Program",
    "sum_group_337": "Farmworker Housing",
    "sum_group_339": "Regional Forest and Fire Capacity",
    "sum_group_341": "Community Assistance for Climate Equity Program",
    "sum_group_345": "Clean Mobility in Schools Project",
    "sum_group_471": "Outreach, Education, and Awareness",
    "sum_group_475": "Fluorinated Gases Emission Reduction Incentives",
    "sum_group_477": "Transition to a Carbon-Neutral Economy",
    "sum_group_479": "Forest Health Research",
    "sum_group_547": "Advanced Technology Demonstration and Pilot Projects",
    "sum_group_549": "Safe and Affordable Drinking Water Fund",
    "sum_group_613": "Low-Carbon Economy Workforce",
    "sum_group_615": "Fire Engines and Maintenance",
    "sum_group_617": "AB617 Implementation Funds",
    "sum_group_619": "Community Composting for Green Spaces Grant Program",
    "sum_group_621": "Community Fire Planning and Preparedness",
    "sum_group_690": "Reuse Grant Program",
    "sum_group_695": "Sustainable Transportation Equity Project",
    "sum_group_765": "Climate Smart Agriculture Technical Assistance Program",
    "sum_group_837": "IDEAL ZEV Workforce Pilot Project",
    "sum_group_839": "SB 1383 Local Assistance Grant Program",
    "sum_group_981": "Co-Digestion Grant Program"
}

# Update the Program_Name field
with arcpy.da.UpdateCursor(Funding_Dist_long, ["Program_Name"]) as cursor:
    for row in cursor:
        if row[0] in column_mapping:
            row[0] = column_mapping[row[0]]  # Replace with the new name
            cursor.updateRow(row)

print("Program_Name values renamed successfully.")

Program_Name values renamed successfully.


### Save both formats as layer packages

In [48]:
# Define paths
Funding_Dist_long = f"{project_gdb}\\Funding_Dist_{suffix}_long"
Funding_Dist_wide = f"{project_gdb}\\Funding_Dist_{suffix}_wide"

layer_long_name = "FeatureLayer_Long"
layer_wide_name = "FeatureLayer_Wide"

FundingDist_lpkx_long = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Data_Restructure\Funding_Dist_long.lpkx"
FundingDist_lpkx_wide = r"C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Data_Restructure\Funding_Dist_wide.lpkx"

# Save 'long' version layer package
try:
    # Step 1: Create a feature layer
    arcpy.management.MakeFeatureLayer(Funding_Dist_long, layer_long_name)
    print(f"Feature layer '{layer_long_name}' created successfully.")

    # Step 2: Create a layer package
    arcpy.management.PackageLayer(layer_long_name, FundingDist_lpkx_long, "PRESERVE", "CONVERT_ARCSDE", "DEFAULT", "ALL")
    print(f"Layer package saved successfully at: {FundingDist_lpkx_long}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Cleanup: Delete the temporary layer
    if arcpy.Exists(layer_long_name):
        arcpy.management.Delete(layer_long_name)
        print(f"Temporary layer '{layer_long_name}' deleted.")

# Save 'wide' version layer package
try:
    # Step 1: Create a feature layer
    arcpy.management.MakeFeatureLayer(Funding_Dist_wide, layer_wide_name)
    print(f"Feature layer '{layer_wide_name}' created successfully.")

    # Step 2: Create a layer package
    arcpy.management.PackageLayer(layer_wide_name, FundingDist_lpkx_wide, "PRESERVE", "CONVERT_ARCSDE", "DEFAULT", "ALL")
    print(f"Layer package saved successfully at: {FundingDist_lpkx_wide}")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Cleanup: Delete the temporary layer
    if arcpy.Exists(layer_wide_name):
        arcpy.management.Delete(layer_wide_name)
        print(f"Temporary layer '{layer_wide_name}' deleted.")



Feature layer 'FeatureLayer_Long' created successfully.
Layer package saved successfully at: C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Data_Restructure\Funding_Dist_long.lpkx
Temporary layer 'FeatureLayer_Long' deleted.
Feature layer 'FeatureLayer_Wide' created successfully.
Layer package saved successfully at: C:\Users\gcowan\Documents\ArcGIS\Projects\Distribution Analysis\Data Restructure\Data_Restructure\Funding_Dist_wide.lpkx
Temporary layer 'FeatureLayer_Wide' deleted.
